In [1]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_lorenz import get_lorenz_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf

# Generate data

In [2]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_lorenz_data(1024, noise_strength=noise_strength)
validation_data = get_lorenz_data(20, noise_strength=noise_strength)

# Set up model and training parameters

In [3]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 3
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = False
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.0
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1024
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [5]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'lorenz_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING


2024-05-20 17:31:38.628830: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


Epoch 0
   training loss 0.034539587795734406, (0.03254698, 1286.3207, 19.857754, 0.6833065)
   validation loss 0.03847106173634529, (0.036848634, 1263.0367, 16.155943, 0.6833065)
decoder loss ratio: 0.198326, decoder SINDy loss  ratio: 1.369341
Epoch 100
   training loss 4.700855060946196e-05, (1.9122857e-05, 0.18385859, 0.097944215, 1.8091271)
   validation loss 6.584195216419175e-05, (2.118638e-05, 0.8072487, 0.26564306, 1.8091271)
decoder loss ratio: 0.000114, decoder SINDy loss  ratio: 0.022515
Epoch 200
   training loss 2.8497168386820704e-05, (6.983829e-06, 0.073698886, 0.026756424, 1.8837696)
   validation loss 3.316579750389792e-05, (7.3224023e-06, 0.3426153, 0.070057, 1.8837696)
decoder loss ratio: 0.000039, decoder SINDy loss  ratio: 0.005938
Epoch 300
   training loss 2.985080936923623e-05, (1.0465412e-05, 0.10094476, 0.018994518, 1.7485946)
   validation loss 3.273372567491606e-05, (1.1413003e-05, 0.23221225, 0.038347766, 1.7485946)
decoder loss ratio: 0.000061, decoder SI

KeyboardInterrupt: 